In [11]:
import os
import sys
import numpy as np
import pandas as pd

Merge data from each dataset

In [12]:
## read archive data
src = open("./archive/nana/train.src", 'r', encoding='utf-8')
tgt = open("./archive/nana/train.tgt", 'r', encoding='utf-8')
nations = dict()
count = np.zeros(1000)
num_nation = 0
info = []
name_list = []
for l1, l2 in zip(src, tgt):
    names = l1[:-1].lower().replace(",", "").split(" ")
    nation = l2[:-1].lower()
    if nation not in nations.keys():
        nations[nation] = num_nation
        num_nation += 1 
    for name in names:
        info.append((name, nations[nation]))
    count[nations[nation]] += 1

src.close()
tgt.close()


In [13]:
## read nationality-classify data
file_list = os.listdir("./archive/datasets")
for fname in file_list:
    nation = fname.replace(".txt", "")
    if nation not in nations.keys():
        nations[nation] = num_nation
        num_nation += 1
    count[nations[nation]] += 1
    with open(os.path.join("./archive/datasets", fname), 'r', encoding='utf-8') as f:
        for l in f:
            names = l1[:-1].lower().split(" ")
            info.append((name, nations[nation]))


In [14]:
## translate nation to region
## na2re[nation_id] -> rid
na2re = []
regions = dict()
with open("./archive/nation-region.txt", 'r', encoding='utf-8') as f:
    num = 0
    for line in f:
        region = line[:-1].split(',')[1]
        if region not in regions.keys():
            regions[region] = num
            num += 1
        na2re.append(regions[region])

In [15]:
## read and write test dataset
src = open("./archive/nana/test.src", 'r', encoding='utf-8')
tgt = open("./archive/nana/test.tgt", 'r', encoding='utf-8')
test_set = open("test_set.txt", 'w', encoding='utf-8')

for l1, l2 in zip(src, tgt):
    names = l1[:-1].lower().replace(",", "")
    nation = l2[:-1].lower()
    test_set.write("%s #%d\n"%(names, na2re[nations[nation]]))

test_set.close()
src.close()
tgt.close()

In [16]:
## write name : nation data
sort_nations = sorted(nations.keys())
# info_set = set(info)
info_set = info
data = open("./data/db.txt", 'w', encoding='utf-8')
for name, nid in info_set:
    if len(name) >= 3 and name.isalpha():
        data.write("%s %d\n"%(name, nid))

with open("./data/nations.txt", 'w', encoding='utf-8') as f:
    for nation in nations.keys():
        f.write("%s\n"%(nation))

data.close()


In [17]:
## read all names
names = []
with open("./data/db.txt", 'r', encoding='utf-8') as f:
    for line in f:
        name = line[:-1].split(",")[0]
        names.append(name)

In [18]:
## translate (name : nation_id) to (name : region_id)
db = open("./data/db.txt", 'r', encoding='utf-8')
redb = open("./data/redb.txt", 'w', encoding='utf-8')

recount = np.zeros(13)

for line in db:
    name, nation = line[:-1].split(" ")
    if name != "mccaughey": 
        ## error name in the data set
        re = na2re[int(nation)]
        recount[re] += 1
        redb.write("%s %d\n"%(name, re))

db.close()
redb.close()

In [19]:
## write region : region_id
with open("./data/regions.txt", 'w', encoding='utf-8') as f:
    for region, rid in regions.items():
        print(rid)
        f.write("%s %d\n"%(region, recount[rid]))

0
1
2
3
4
5
6
7
8
9
10
11
12
